In [14]:
import collections
import math
import random
import sys
import time
import os
import numpy as np
import torch
from torch import nn
import torch.utils.data as Data
#import d21zh_pytorch as d21


In [10]:
with open("data/ptb/ptb.train.txt","r") as f:
    lines=f.readlines()
    raw_dataset=[st.split() for st in lines]

len(raw_dataset)

42068

In [11]:
#建立词语索引
counter=collections.Counter([ tk for st in raw_dataset for tk in st])
counter=dict(filter(lambda x : x[1]>=5,counter.items()))

In [23]:
idx_to_token=[tk for tk,_ in counter.items()]
token_to_idx={tk:idx for idx,tk in enumerate(idx_to_token)}
dataset=[[token_to_idx[tk] for tk in st if tk in token_to_idx] for st in raw_dataset]
num_tokens=sum([len(st) for st in dataset])
num_tokens

887100

In [25]:
#二次采样 丢弃高频词汇
def discard(idx):
    return random.uniform(0,1)<1-math.sqrt(1e-4/counter[idx_to_token[idx]]*num_tokens)

subsampled_dataset=[[tk for tk in st if not discard(tk)] for st in dataset]
sum([len(st) for st in subsampled_dataset])

375682

In [32]:
#比较一个词在二次采样前后在数据集出现的情况
def compare_counts(token):
    before=sum([st.count(token_to_idx[token]) for st in dataset])
    after=sum([st.count(token_to_idx[token]) for st in subsampled_dataset])
    print("before=%d,after=%d ",(before,after))
compare_counts('join')

before=%d,after=%d  (45, 45)


In [35]:
#提取中心词和背景词
def get_centers_and_contexts(dataset,max_window_size):
    centers,contexts=[],[]
    for st in dataset:
        if len(st)<2:
            continue
        centers+=st
        for center_i in range(len(st)):
            window_size=random.randint(1,max_window_size)
            indices=list(range(max(0,center_i-window_size),min(len(st),center_i+1+window_size)))
            indices.remove(center_i)
            contexts.append([st[idx] for idx in indices])
        return centers,contexts

dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1]
center 1 has contexts [0, 2]
center 2 has contexts [1, 3]
center 3 has contexts [2, 4]
center 4 has contexts [3, 5]
center 5 has contexts [4, 6]
center 6 has contexts [4, 5]
